# Code for the regression for the impact of the flow of momentum on managerial decision-making

In [ ]:
#import the pitch control and PCV file
import numpy as np
import pandas as pd
import scipy.stats as stats
import pickle
from progressbar import ProgressBar
from bevel.linear_ordinal_regression import OrderedLogit
infile = open('input PC regression Momentum on decisions.p','rb')
df_mom = pickle.load(infile)
infile.close()

In [ ]:
#create additional columns with ordinal values for the decisions
decisions = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['Type of decision'] == 'neutral':
        decisions.append(0)
    elif df_mom.iloc[i]['Type of decision'] == 'offensive':
        decisions.append(1)
    else:
        decisions.append(-1)
        
df_mom['ordinal decisions'] = decisions

In [ ]:
#create additional columns with the score difference for both teams (match state)
home_difference = []
away_difference = []
for i in range(len(df_mom)):
    home_difference.append(df_mom.iloc[i]['Home score'] - df_mom.iloc[i]['Away score'])
    away_difference.append(df_mom.iloc[i]['Away score'] - df_mom.iloc[i]['Home score'])
    
df_mom['home difference'] = home_difference
df_mom['away difference'] = away_difference

In [ ]:
#create additional columns with the pitch control and PCV (WPC) values, and the score difference depending on the team which made the decision
final_pc = []
final_wpc = []
final_difference = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['team'] == 'Home':
        final_pc.append(df_mom.iloc[i]['PC home 10min before'])
        final_wpc.append(df_mom.iloc[i]['WPC home 10min before'])
        final_difference.append(df_mom.iloc[i]['home difference'])
    else:
        final_pc.append(df_mom.iloc[i]['PC away 10min before'])
        final_wpc.append(df_mom.iloc[i]['WPC away 10min before'])
        final_difference.append(df_mom.iloc[i]['away difference'])
df_mom['PC team'] = final_pc
df_mom['WPC team'] = final_wpc
df_mom['score difference'] = final_difference

In [ ]:
#normalize the PCV values
maximum = df_mom['WPC team'].max()
minimum = df_mom['WPC team'].min()
df_mom['WPC team'] = (df_mom['WPC team'] - minimum)/(maximum-minimum)

In [ ]:
#only select the relevant columns in the dataframe
df = pd.DataFrame(df_mom,columns=['PC team', 'WPC team', 'Time left', 'ordinal decisions', 'score difference', 'offensive', 'defensive', 'neutral'])
df = df.dropna()

In [ ]:
#regression for pitch control and no moderators
from statistics import mean
p_valuepc = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    ordinal_decisions = []
    from bevel.linear_ordinal_regression import OrderedLogit
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg['pc before']
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    pc_param.append(model.beta_[0])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
       'beta pc': pc_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pc_without_moderators = pd.DataFrame(data)
     

In [ ]:
#regression for pitch control and score difference (match state) as moderator
from statistics import mean
p_valuepc = []
p_valuesd = []
sd_param = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    score_difference = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        score_difference.append(df_new.iloc[j]['score difference'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions,
           'score difference': score_difference}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg[['pc before', 'score difference']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    p_valuesd.append(model.p_values_[3])
    pc_param.append(model.beta_[0])
    sd_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
        'p_value sd': p_valuesd,
       'beta pc': pc_param,
        'beta sd': sd_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pc_with_score_difference = pd.DataFrame(data)
     

In [ ]:
#regression for pitch control and time left as moderator
from statistics import mean
p_valuepc = []
p_valuetl = []
tl_param = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    time = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        time.append(df_new.iloc[j]['Time left'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions,
           'Time left': time}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg[['pc before', 'Time left']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    p_valuetl.append(model.p_values_[3])
    pc_param.append(model.beta_[0])
    tl_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
        'p_value tl': p_valuetl,
       'beta pc': pc_param,
        'beta tl': tl_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pc_with_time_left = pd.DataFrame(data)
     

In [ ]:
#regression for PCV and no moderators
from statistics import mean
p_valuepc = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    ordinal_decisions = []
    from bevel.linear_ordinal_regression import OrderedLogit
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg['wpc before']
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    pc_param.append(model.beta_[0])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
       'beta pc': pc_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pcv_without_moderators = pd.DataFrame(data)
     

In [ ]:
#regression for PCV and score difference (match state) as moderator
from statistics import mean
p_valuepc = []
p_valuesd = []
sd_param = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    score_difference = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        score_difference.append(df_new.iloc[j]['score difference'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions,
           'score difference': score_difference}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg[['wpc before', 'score difference']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    p_valuesd.append(model.p_values_[3])
    pc_param.append(model.beta_[0])
    sd_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
        'p_value sd': p_valuesd,
       'beta pc': pc_param,
        'beta sd': sd_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pcv_with_score_difference = pd.DataFrame(data)
     

In [ ]:
#regression for PCV and time left as moderator
from statistics import mean
p_valuepc = []
p_valuetl = []
tl_param = []
pc_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    pc_before = []
    wpc_before = []
    time = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        pc_before.append(df_new.iloc[j]['PC team'])
        wpc_before.append(df_new.iloc[j]['WPC team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        time.append(df_new.iloc[j]['Time left'])

    #create dataframe with the necessary data for regression
    data = {'pc before': pc_before,
           'wpc before': wpc_before,
           'ordinal decisions': ordinal_decisions,
           'Time left': time}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg[['wpc before', 'Time left']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuepc.append(model.p_values_[0])
    p_valuetl.append(model.p_values_[3])
    pc_param.append(model.beta_[0])
    tl_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value pc': p_valuepc,
        'p_value tl': p_valuetl,
       'beta pc': pc_param,
        'beta tl': tl_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_pcv_with_time_left = pd.DataFrame(data)
     

In [ ]:
#import xT file
import pickle
infile = open('input xT regression Momentum on decisions.p','rb')
df_mom = pickle.load(infile)
infile.close()

In [ ]:
#create additional columns with ordinal values for the decisions
decisions = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['Type of decision'] == 'neutral':
        decisions.append(0)
    elif df_mom.iloc[i]['Type of decision'] == 'offensive':
        decisions.append(1)
    else:
        decisions.append(-1)
        
df_mom['ordinal decisions'] = decisions

In [ ]:
#create additional columns with the score difference for both teams (match state)
home_difference = []
away_difference = []
for i in range(len(df_mom)):
    home_difference.append(df_mom.iloc[i]['Home score'] - df_mom.iloc[i]['Away score'])
    away_difference.append(df_mom.iloc[i]['Away score'] - df_mom.iloc[i]['Home score'])
    
df_mom['home difference'] = home_difference
df_mom['away difference'] = away_difference

In [ ]:
#create additional columns with the xT values, and the score difference depending on the team which made the decision
final_xt = []
final_difference = []
for i in range(len(df_mom)):
    if df_mom.iloc[i]['team'] == 'Home':
        final_xt.append(df_mom.iloc[i]['xT_prev_10min_home'])
        final_difference.append(df_mom.iloc[i]['home difference'])
    else:
        final_xt.append(df_mom.iloc[i]['xT_prev_10min_away'])
        final_difference.append(df_mom.iloc[i]['away difference'])
df_mom['xT team'] = final_xt
df_mom['score difference'] = final_difference

In [ ]:
#normalize the xT values
maximum = df_mom['xT team'].max()
minimum = df_mom['xT team'].min()
df_mom['xT team'] = (df_mom['xT team'] - minimum)/(maximum-minimum)

In [ ]:
#only select the relevant columns in the dataframe
df = pd.DataFrame(df_mom,columns=['xT team', 'Time left', 'ordinal decisions', 'score difference', 'offensive', 'defensive', 'neutral'])
df = df.dropna()

In [ ]:
#regression for xT and no moderator
from statistics import mean
p_valuext = []
xt_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    xt_before = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        xt_before.append(df_new.iloc[j]['xT team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])

    #create dataframe with the necessary data for regression
    data = {'xT before': xt_before,
           'ordinal decisions': ordinal_decisions}
    
    df_reg = pd.DataFrame(data)
    
    #Execute the regression
    X = df_reg['xT before']
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuext.append(model.p_values_[0])
    xt_param.append(model.beta_[0])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value xt': p_valuext,
       'beta xt': xt_param,
       'somers d': somers_d,
       'SE': se}

#final dataframe with the statistics this regression
df_xt_without_moderators = pd.DataFrame(data)
     

In [ ]:
#regression for xT and score difference (match state) as moderator
from statistics import mean
p_valuext = []
p_valuesd = []
sd_param = []
xt_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    xt_before = []
    score_difference = []
    ordinal_decisions = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions     
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        xt_before.append(df_new.iloc[j]['xT team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        score_difference.append(df_new.iloc[j]['score difference'])

    #create dataframe with the necessary data for regression        
    data = {'xT before': xt_before,
           'ordinal decisions': ordinal_decisions,
           'score difference': score_difference}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression    
    X = df_reg[['xT before', 'score difference']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuext.append(model.p_values_[0])
    p_valuesd.append(model.p_values_[3])
    xt_param.append(model.beta_[0])
    sd_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value xt': p_valuext,
        'p_value sd': p_valuesd,
       'beta xt': xt_param,
        'beta sd': sd_param,
       'somers d': somers_d,
       'SE': se}
#final dataframe with the statistics this regression
df_xt_with_score_difference = pd.DataFrame(data)
     

In [ ]:
#regression for xT and time left as moderator
from statistics import mean
p_valuext = []
p_valuetl = []
tl_param = []
xt_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    ordinal_decisions = []
    xt_before = []
    time = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions      
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        xt_before.append(df_new.iloc[j]['xT team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        time.append(df_new.iloc[j]['Time left'])

    #create dataframe with the necessary data for regression     
    data = {'xT before': xt_before,
           'ordinal decisions': ordinal_decisions,
           'Time left': time}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression 
    X = df_reg[['xT before', 'Time left']]
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuext.append(model.p_values_[0])
    p_valuetl.append(model.p_values_[3])
    xt_param.append(model.beta_[0])
    tl_param.append(model.beta_[1])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value xt': p_valuext,
        'p_value tl': p_valuetl,
       'beta xt': xt_param,
        'beta tl': tl_param,
       'somers d': somers_d,
       'SE': se}
#final dataframe with the statistics this regression
df_xt_with_time_left = pd.DataFrame(data)
     

In [ ]:
#regression for time left
from statistics import mean
p_valuetl = []
tl_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    ordinal_decisions = []
    xt_before = []
    time = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions  
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        xt_before.append(df_new.iloc[j]['xT team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        time.append(df_new.iloc[j]['Time left'])

    #create dataframe with the necessary data for regression 
    data = {'xT before': xt_before,
           'ordinal decisions': ordinal_decisions,
           'Time left': time}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression     
    X = df_reg['Time left']
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuetl.append(model.p_values_[0])
    tl_param.append(model.beta_[0])
    se.append(mean(model.se_))
    somers_d.append(model.score_)
data = {'p_value tl': p_valuetl,
        'beta tl': tl_param,
       'somers d': somers_d,
       'SE': se}
#final dataframe with the statistics this regression
df_tl = pd.DataFrame(data)
     

In [ ]:
#regression for score difference (match state)
from statistics import mean
p_valuesd = []
sd_param = []
somers_d = []
se = []
pbar = ProgressBar()
#iterate the regression over a hundred times
for i in pbar(range(100)):
    ordinal_decisions = []
    xt_before = []
    sd = []
    df_offensive = df.loc[df['offensive'] == 1]
    df_defensive = df.loc[df['defensive'] == 1]
    df_neutral = df.loc[df['neutral'] == 1]
    #take 80% random samples of the types of decisins to solve the imblanced decisions 
    df_offensive = df_offensive.sample(frac=0.8, replace=True)
    df_defensive = df_defensive.sample(frac=0.8, replace=True)
    df_neutral = df_neutral.sample(frac=0.8, replace=True)
    df_new = pd.concat([df_offensive, df_defensive, df_neutral])
    for j in range(len(df_new)):
        xt_before.append(df_new.iloc[j]['xT team'])
        ordinal_decisions.append(df_new.iloc[j]['ordinal decisions'])
        sd.append(df_new.iloc[j]['score difference'])

    #create dataframe with the necessary data for regression 
    data = {'xT before': xt_before,
           'ordinal decisions': ordinal_decisions,
           'sd': sd}
    df_reg = pd.DataFrame(data)
    
    #Execute the regression  
    X = df_reg['sd']
    y = df_reg['ordinal decisions']

    ol = OrderedLogit()
    model = ol.fit(X, y)

    #Create a dataframe with the important statistics
    p_valuesd.append(model.p_values_[0])
    sd_param.append(model.beta_[0])
    se.append(mean(model.se_))
    somers_d.append(model.score_)

data = {'p_value sd': p_valuesd,
        'beta sd': sd_param,
       'somers d': somers_d,
       'SE': se}
#final dataframe with the statistics this regression
df_sd = pd.DataFrame(data)
     